In [ ]:
import os
import datetime
import cv2

############## user define #############
ImageSize = 32
DatasetType = ['mnist', 'cifar-10']
DatasetIndex = 1
epoch = 200
netName = 'VGG11ave'
binary = True
#####################################

DatasetName = DatasetType[DatasetIndex] + '_' + str(ImageSize)
ProjectName = 'Project_' + DatasetName + '_' + str(datetime.date.today())

def prcess(file_path):
    im = cv2.imread(file_path, 0)
    ret,thresh1 = cv2.threshold(im,127,255,cv2.THRESH_BINARY)
    cv2.imwrite(file_path, ret)

def recursive_file_check(path):
    if os.path.isdir(path):
        files = os.listdir(path)
        for file in files:
            recursive_file_check(path + "\\" + file)
    else:
        process(path)

%cd /content
if (os.path.exists('/content/CUINNESS')==False):
    !git clone https://github.com/knmrtkt/CUINNESS.git
else:
    !git pull
%cd CUINNESS
!git pull

%cd /content
if (DatasetType[DatasetIndex] == 'mnist'):
    if (os.path.exists('/content/mnist_png')==False):
        !git clone https://github.com/myleott/mnist_png.git
        %cd mnist_png
        !tar -zxvf mnist_png.tar.gz
        !rsync -a mnist_png/testing/ mnist_png/training
        if (binary == True):
            recursive_file_check('mnist_png/training/')
        !ls -1 -d ../mnist_png/mnist_png/training/* > list_tmp1.txt
        !ls -1 ../mnist_png/mnist_png/training > list_tmp2.txt
        !paste -d ' ' list_tmp1.txt list_tmp2.txt > ../CUINNESS/list.txt
        !rm list_tmp1.txt list_tmp2.txt

elif (DatasetType[DatasetIndex] == 'cifar-10'):
    if (os.path.exists('/content/cifar2png')==False):
        !git clone https://github.com/knjcode/cifar2png.git
        %cd cifar2png
        !python3 cifar2png cifar10 cifar10png --name-with-batch-index
        !rsync -a cifar10png/test/ cifar10png/train
        %cd /content/CUINNESS
        !ls -1 -d ../cifar2png/cifar10png/train/* > list_tmp1.txt
        !ls -1 ../cifar2png/cifar10png/train > list_tmp2.txt
        !paste -d ' ' list_tmp1.txt list_tmp2.txt > ../CUINNESS/list.txt
        !rm list_tmp1.txt list_tmp2.txt

%cd /content/CUINNESS
if (os.path.exists(DatasetName + '_dataset.pkl')==False):
    !python3 gen_training_data.py --pathfile list.txt --dataset $DatasetName --size $ImageSize --keepaspect yes
!python3 guinness.py $ProjectName $DatasetName $epoch --netName=$netName
%cd $ProjectName
!python3 eval.py --dataset ../$DatasetName --size $ImageSize
%cd ..
ProjectZipPath = '/content/CUINNESS/' + ProjectName + '.zip'
!zip -r $ProjectZipPath /content/CUINNESS/$ProjectName
from google.colab import files
files.download(ProjectZipPath)